# Explore BL Sessions Data

Here we introduce some useful DataJoint tools to explore the bl19 pipeline

First thing first, import datajoint

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import utility.blob_transformation as bt
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from matplotlib import cm

import pandas as pd

import bl_pipeline.acquisition as acquisition
import bl_pipeline.shadow.acquisition as shadow_acquisition


[2022-08-23 09:55:10,123][INFO]: Connecting alvaros@datajoint01.pni.princeton.edu:3306
[2022-08-23 09:55:10,600][INFO]: Connected alvaros@datajoint01.pni.princeton.edu:3306


In [3]:
bdata = dj.create_virtual_module('bdatatest', 'bdatatest')

In [4]:
dj.blob.use_32bit_dims = True

## Get handle to DBs

lab         =  researchers & rig DB <br />
subject     =  rats DB <br />
acquisition =  sessions DB

In [5]:
import traceback
dj.blob.bypass_serialization = True

sas = shadow_acquisition.Sessions

acquisition_shadow_fields = pd.DataFrame.from_dict(sas.heading.attributes, orient='index')
acquisition_shadow_fields = acquisition_shadow_fields['name'].tolist()
acquisition_shadow_fields.remove('peh')
acquisition_shadow_fields.remove('protocol_data')

sas = sas.proj(*acquisition_shadow_fields) & 'session_date > "2022-08-22" and session_starttime > "05:53:12"'

bdata_t = bdata.Sessions.proj('protocol_data', 'sessiondate', 'starttime') * bdata.ParsedEvents() & 'sessiondate > "2022-08-22"  and starttime > "05:53:12"'
bdata_t = bdata_t.proj('protocol_data', 'peh')
final_table = sas * bdata_t


q_insert = final_table - acquisition.Sessions2.proj()


for t in (q_insert).fetch(as_dict=True):
    try:
        acquisition.Sessions2.insert1(t)
    except Exception:
        print("Error when inserting {}".format(t))
        traceback.print_exc()

In [7]:
dj.blob.bypass_serialization = False
session_data = acquisition.Sessions2.fetch(as_dict=True)


protocol_data_python = bt.transform_blob(session_data[0]['peh'])
df_protocol_data = pd.DataFrame(protocol_data_python)

df_protocol_data

,pokes,waves,states
0,"{'C': [2951.837104, 2951.966304], 'L': [], 'R'...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 2949.059604], [2952.992304,..."
1,"{'C': [2979.848204, 2979.967604], 'L': [[2962....","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 2957.147504], [2980.9936040..."
2,"{'C': [2988.736504, 2990.200304], 'L': [[2990....","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 2984.376104], [2991.226304,..."
3,"{'C': [[3003.1977039999997, 3003.2672039999998...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 2994.405504], [3004.293204,..."
4,"{'C': [3008.6075039999996, 3010.2285039999997]...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 3007.333104], [3013.5091039..."
...,...,...,...
420,"{'C': [8544.312502, 8546.131002], 'L': [], 'R'...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 8543.094402], [8549.593402,..."
421,"{'C': [8551.460802, 8553.186402], 'L': [[8553....","{'direct_left_reward': [8553.675502, 8553.8289...","{'state_0': [[nan, 8550.269502], [8558.6086019..."
422,"{'C': [8561.340117000002, 8563.315217000001], ...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 8558.609517], [8569.778617,..."
423,"{'C': [8575.647916, 8577.580616000001], 'L': [...","{'direct_left_reward': [], 'direct_right_rewar...","{'state_0': [[nan, 8569.779516], [8580.8967160..."


In [6]:
peh = bt.transform_blob(parsed_events[0]['peh'])
df_peh = pd.DataFrame(peh)
df_peh

,pokes,waves,states
0,"{'C': [[1005.2353039999999, 1005.2579039999999...","{'direct_reward': [], 'stimulatorwave': [1005....","{'state_0': [[nan, 883.515104], [1008.358804, ..."
1,"{'C': [1010.399115, 1012.049215], 'L': [[1008....","{'direct_reward': [1012.223015, 1012.327515], ...","{'state_0': [[nan, 1008.359015], [1012.828215,..."
2,"{'C': [1015.987304, 1017.469004], 'L': [[1017....","{'direct_reward': [1017.671604, 1017.776204], ...","{'state_0': [[nan, 1013.765804], [1018.2769040..."
3,"{'C': [1020.607404, 1022.716804], 'L': [], 'R'...","{'direct_reward': [], 'stimulatorwave': [1021....","{'state_0': [[nan, 1019.300804], [1026.878104,..."
4,"{'C': [[1027.853115, 1027.8705149999998], [102...","{'direct_reward': [1030.669915, 1030.799014999...","{'state_0': [[nan, 1026.878315], [1031.2997149..."
5,"{'C': [[1033.070804, 1033.0829039999999], [103...","{'direct_reward': [], 'stimulatorwave': [1033....","{'state_0': [[nan, 1031.845504], [1039.5069039..."
